In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

import kaggle

!kaggle kernels output mohamedzidan136/t5v2cnndailymail -p /content/simplet5

In [ ]:
!pip install torch
!pip install pytorch-pretrained-bert
!pip install sentence-transformers
!pip install Rouge
!pip install dataset
!pip install evaluate
!pip install transformers
#!pip install bert-tensorflow

In [ ]:
#!pip install dash==1.20.0 

In [ ]:
#!pip install --upgrade simplet5

In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def bertSent_embeding(sentences):
   
    ## Add sentence head and tail as BERT requested
    marked_sent = ["[CLS] " +item + " [SEP]" for item in sentences]
    
    tokenized_sent = [tokenizer.tokenize(item ) for item in marked_sent]
    

    
    ## index to BERT vocabulary
    indexed_tokens = [tokenizer.convert_tokens_to_ids(item) for item in tokenized_sent]
    tokens_tensor = [torch.tensor([item]) for item in indexed_tokens]

    ## add segment id as BERT requested
    segments_ids = [[1] * len(item) for ind,item in enumerate(tokenized_sent)]
    segments_tensors = [torch.tensor([item]) for item in segments_ids]
    ## load BERT base model and set to evaluation mode
    ##### bert_model = SentenceTransformer('stsb-bert-large')


    #model.eval() is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. 
    #For example, Dropouts Layers, BatchNorm Layers etc.
    #You need to turn them off during model evaluation, and .eval() will do it for you. In addition, 
    #the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation:
    bert_model.eval()
    
    ## Output 12 layers of latent vector
    assert len(tokens_tensor) == len(segments_tensors)
    encoded_layers_list = []
    for i in range(len(tokens_tensor)):
        with torch.no_grad():
            encoded_layers, _ = bert_model(tokens_tensor[i], segments_tensors[i])
        encoded_layers_list.append(encoded_layers)
    
    ## Use only the last layer vetcor **we can use others**
  
    token_vecs_list =[layers[11][0] for layers in encoded_layers_list]
    ## Pooling word vector to sentence vector, use mean pooling
    sentence_embedding_list = [torch.mean(vec, dim=0).numpy() for vec in token_vecs_list]

    return sentence_embedding_list

100%|██████████| 407873900/407873900 [00:34<00:00, 11744754.50B/s]


In [ ]:
def kmeans_sumIndex(sentence_embedding_list):
  
    try:
      n_clusters =5
      #np.ceil(len(sentence_embedding_list)**0.5)
      kmeans = KMeans(n_clusters=int(n_clusters))
      kmeans = kmeans.fit(sentence_embedding_list)
      #Compute minimum distances between one point and a set of points.
      #get centers and compare with sentence_embedding by bert !!!!("centers is the most common sentences")
      sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
      sum_index = sorted(sum_index)
      return sum_index
    except:
      n_clusters =1
      #np.ceil(len(sentence_embedding_list)**0.5)
      kmeans = KMeans(n_clusters=int(n_clusters))
      kmeans = kmeans.fit(sentence_embedding_list)
      #Compute minimum distances between one point and a set of points.
      #get centers and compare with sentence_embedding by bert !!!!("centers is the most common sentences")
      sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
      sum_index = sorted(sum_index)
      return sum_index

In [ ]:
def bertSummarize(text):
  
    sentences = sent_tokenize(text)

    sentence_embedding_list = bertSent_embeding(sentences)

    sum_index = kmeans_sumIndex(sentence_embedding_list)
    
    summary = ' '.join([sentences[ind] for ind in sum_index])
   
    return summary

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

T5_PATH = '/content/simplet5/outputs/best_model' 
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH)

In [ ]:
text = """Off-Policy reinforcement learning has been a
driving force for the state-of-the-art conversational AIs leading to more natural humanagent interactions and improving the user satisfaction for goal-oriented agents. However,
in large-scale commercial settings, it is often challenging to balance between policy improvements and experience continuity on the
broad spectrum of applications handled by
such system. In the literature, off-policy evaluation and guard-railing on aggregate statistics
has been commonly used to address this problem. In this paper, we propose a method for curating and leveraging high-precision samples
sourced from historical regression incident reports to validate, safe-guard, and improve policies prior to the online deployment. We conducted extensive experiments using data from
a real-world conversational system and actual
regression incidents. The proposed method is
currently deployed in our production system to
protect customers against broken experiences
and enable long-term policy improvements."""

In [ ]:
text

'Off-Policy reinforcement learning has been a\ndriving force for the state-of-the-art conversational AIs leading to more natural humanagent interactions and improving the user satisfaction for goal-oriented agents. However,\nin large-scale commercial settings, it is often challenging to balance between policy improvements and experience continuity on the\nbroad spectrum of applications handled by\nsuch system. In the literature, off-policy evaluation and guard-railing on aggregate statistics\nhas been commonly used to address this problem. In this paper, we propose a method for curating and leveraging high-precision samples\nsourced from historical regression incident reports to validate, safe-guard, and improve policies prior to the online deployment. We conducted extensive experiments using data from\na real-world conversational system and actual\nregression incidents. The proposed method is\ncurrently deployed in our production system to\nprotect customers against broken experiences

In [ ]:
ext_summary= bertSummarize(text)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
ext_summary

'Off-Policy reinforcement learning has been a\ndriving force for the state-of-the-art conversational AIs leading to more natural humanagent interactions and improving the user satisfaction for goal-oriented agents. However,\nin large-scale commercial settings, it is often challenging to balance between policy improvements and experience continuity on the\nbroad spectrum of applications handled by\nsuch system. In the literature, off-policy evaluation and guard-railing on aggregate statistics\nhas been commonly used to address this problem. In this paper, we propose a method for curating and leveraging high-precision samples\nsourced from historical regression incident reports to validate, safe-guard, and improve policies prior to the online deployment. We conducted extensive experiments using data from\na real-world conversational system and actual\nregression incidents.'

In [ ]:
inputs = t5_tokenizer.encode("summarize:" + ext_summary, return_tensors="pt", max_length=512, padding='max_length', truncation=True)

In [ ]:
summary_ids = t5_model.generate(inputs,

                                    num_beams=int(2),

                                    no_repeat_ngram_size=3,

                                    length_penalty=2.0,

                                    min_length=100,

                                    max_length=200,

                                    early_stopping=True)

output = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)


In [ ]:
output

"Off-Policy reinforcement learning has been a driving force for conversational AIs. In large-scale commercial settings, it is often difficult to balance between policy improvements and experience continuity. We conducted extensive experiments using data from historical regression incident reports. This paper propose a method for curating and leveraging high-precision samples from historical. regression incidents to validate, safe-guard, and improve policies prior to the online deployment. 'It is often challenging to balance policy improvements with experience continuity on the broad spectrum of applications handled by such systems'"